# Analysing gpt-4o model vs finetuned:

In [ ]:
import glob
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from mgeval import core, utils
from sklearn.model_selection import LeaveOneOut

In [ ]:
MODEL_1 = "gpt-4o"
MODEL_2 = "finetuned"

SET_1_PATH = "data/evaluation/midi/{}/*".format(MODEL_1)
SET_2_PATH = "data/evaluation/midi/{}/*".format(MODEL_2)

SET_1_CALCULATED_METRICS_PATH = "data/evaluation/results/{}_vs_{}/{}_calculated_metrics.pkl".format(MODEL_1, MODEL_2, MODEL_1)
SET_2_CALCULATED_METRICS_PATH = "data/evaluation/results/{}_vs_{}/{}_calculated_metrics.pkl".format(MODEL_1, MODEL_2, MODEL_2)

ABSOLUTE_METRICS_STATISTICS_PATH = "data/evaluation/results/{}_vs_{}/absolute_metrics_statistics.pkl".format(MODEL_1, MODEL_2)
RELATIVE_METRICS_STATISTICS_PATH = "data/evaluation/results/{}_vs_{}/relative_metrics_statistics.pkl".format(MODEL_1, MODEL_2)

## Absolute measurement: statistic analysis


Assign sample dataset path

In [ ]:
set1 = glob.glob(SET_1_PATH)
print(set1)

Assign baseline dataset path

In [ ]:
set2 = glob.glob(SET_2_PATH)
print(set2)

### Build empty dictionary to fill in measurement across samples

In [ ]:
num_samples = min(len(set2), len(set1))
display(num_samples)

In [ ]:
def init_eval_dict(num_samples):
    return {
        # Pitch-based features
        'total_used_pitch': np.zeros((num_samples, 1)),
        'total_pitch_class_histogram': np.zeros((num_samples, 12)),
        'pitch_class_transition_matrix': np.zeros((num_samples, 12, 12)),
        'pitch_range': np.zeros((num_samples, 1)),
        'avg_pitch_shift': np.zeros((num_samples, 1)),
        # Rhythm-based features
        'total_used_note': np.zeros((num_samples, 1)),
        'avg_IOI': np.zeros((num_samples, 1)),
        'note_length_hist': np.zeros((num_samples, 12)),
        'note_length_transition_matrix': np.zeros((num_samples, 12, 12)),
    }

set1_eval = init_eval_dict(num_samples)
set2_eval = init_eval_dict(num_samples)

List of metrics to calculate

In [ ]:
metrics_list = set1_eval.keys()

### Calculate absolute metrics

First dataset

In [ ]:
for i in range(0, num_samples):
    feature = core.extract_feature(set1[i])
    for metric in metrics_list:
        set1_eval[metric][i] = getattr(core.metrics(), metric)(feature)

Write set1_eval calculated metrics

In [ ]:
with open(SET_1_CALCULATED_METRICS_PATH, "wb") as f:
    pickle.dump(set1_eval, f)

Second dataset

In [ ]:
for i in range(0, num_samples):
    feature = core.extract_feature(set2[i])
    for metric in metrics_list:
        set2_eval[metric][i] = getattr(core.metrics(), metric)(feature)

Write set2_eval calculated metrics

In [ ]:
with open(SET_2_CALCULATED_METRICS_PATH, "wb") as f:
    pickle.dump(set2_eval, f)

Statistic analysis: absolute measurement

In [ ]:
summary_stats = {}
for metric in metrics_list:
    set1_mean = np.mean(set1_eval[metric], axis=0)
    set1_std = np.std(set1_eval[metric], axis=0)
    set2_mean = np.mean(set2_eval[metric], axis=0)
    set2_std = np.std(set2_eval[metric], axis=0)
    summary_stats[metric] = {
        MODEL_1: {'mean': set1_mean.tolist(), 'std': set1_std.tolist()},
        MODEL_2: {'mean': set2_mean.tolist(), 'std': set2_std.tolist()}
    }

    print('------------------------------------------------------------------------------------------------')
    print('Metric: {}'.format(metric))
    print('{} => Mean: {}, Std: {}'.format(MODEL_1, set1_mean, set1_std))
    print('{} => Mean: {}, Std: {}'.format(MODEL_2, set2_mean, set2_std))
    print('------------------------------------------------------------------------------------------------\n')

Write absolute metrics statistics

In [ ]:
with open(ABSOLUTE_METRICS_STATISTICS_PATH, "wb") as f:
    pickle.dump(summary_stats, f)

## Relative measurement: generalizes the result among features with various dimensions


the features are sum- marized to 
- the intra-set distances
- the difference of intra-set and inter-set distances.

exhaustive cross-validation for intra-set distances measurement

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(np.arange(num_samples))
set1_intra = np.zeros((num_samples, len(metrics_list), num_samples - 1))
set2_intra = np.zeros((num_samples, len(metrics_list), num_samples - 1))
for i in range(len(metrics_list)):
    for train_index, test_index in loo.split(np.arange(num_samples)):
        set1_intra[test_index[0]][i] = utils.c_dist(set1_eval[metrics_list[i]][test_index], set1_eval[metrics_list[i]][train_index])
        set2_intra[test_index[0]][i] = utils.c_dist(set2_eval[metrics_list[i]][test_index], set2_eval[metrics_list[i]][train_index])

exhaustive cross-validation for inter-set distances measurement

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(np.arange(num_samples))
sets_inter = np.zeros((num_samples, len(metrics_list), num_samples))

for i in range(len(metrics_list)):
    for train_index, test_index in loo.split(np.arange(num_samples)):
        sets_inter[test_index[0]][i] = utils.c_dist(set1_eval[metrics_list[i]][test_index], set2_eval[metrics_list[i]])

visualization of intra-set and inter-set distances

In [ ]:
plot_set1_intra = np.transpose(set1_intra,(1, 0, 2)).reshape(len(metrics_list), -1)
plot_set2_intra = np.transpose(set2_intra,(1, 0, 2)).reshape(len(metrics_list), -1)
plot_sets_inter = np.transpose(sets_inter,(1, 0, 2)).reshape(len(metrics_list), -1)
for i in range(0,len(metrics_list)):
    sns.kdeplot(plot_set1_intra[i], label='intra_set1')
    sns.kdeplot(plot_sets_inter[i], label='inter')
    sns.kdeplot(plot_set2_intra[i], label='intra_set2')

    plt.title(metrics_list[i])
    plt.xlabel('Euclidean distance')
    plt.show()

the difference of intra-set and inter-set distances.

In [ ]:
relative_stats = {}
for i in range(0, len(metrics_list)):
    kl1 = utils.kl_dist(plot_set1_intra[i], plot_sets_inter[i])
    overlap1 = utils.overlap_area(plot_set1_intra[i], plot_sets_inter[i])
    kl2 = utils.kl_dist(plot_set2_intra[i], plot_sets_inter[i])
    overlap2 = utils.overlap_area(plot_set2_intra[i], plot_sets_inter[i])
    metric = metrics_list[i]
    relative_stats[metric] = {
        MODEL_1: {
            'kl_divergence': kl1,
            'overlap_area': overlap1
        },
        MODEL_2: {
            'kl_divergence': kl2,
            'overlap_area': overlap2
        }
    }
    print('------------------------------------------------------------------------------------------------')
    print('Metric: {}'.format(metric))
    print('{} => Kullback–Leibler divergence: {}, Overlap area: {}'.format(MODEL_1, kl1, overlap1))
    print('{} => Kullback–Leibler divergence: {}, Overlap area: {}'.format(MODEL_2, kl2, overlap2))
    print('------------------------------------------------------------------------------------------------\n')

Write relative metrics statistics

In [ ]:
with open(RELATIVE_METRICS_STATISTICS_PATH, "wb") as f:
    pickle.dump(relative_stats, f)